In [134]:
#数据装载
import csv
import numpy as np
import pandas as pd

df = pd.read_csv('train_data.csv')
data = np.array(df)
print data

[['A1' 'A3' 'B7' 'B6' 'A2' 4000]
 ['B1' 'A3' 'B7' 'D6' 'B2' 3000]
 ['B1' 'A3' 'B7' 'A6' 'A2' 6000]
 ...
 ['B1' 'B3' 'D7' 'A6' 'A2' 10000]
 ['C1' 'C3' 'A7' 'C6' 'A2' 3000]
 ['D1' 'B3' 'B7' 'A6' 'A2' 17000]]


In [135]:
#预测值分层
for i in range(len(data)):
    if 1000<=data[i][5]<3000:
        data[i][5] = 1000
    if 3000<=data[i][5]<5000:
        data[i][5] = 3000
    if 5000<=data[i][5]<8000:
        data[i][5] = 5000
    if 8000<=data[i][5]<10000:
        data[i][5] = 8000
    if 10000<=data[i][5]<40000:
        data[i][5] = 10000
    if data[i][5]>=40000:
        data[i][5] = 40000

print data

[['A1' 'A3' 'B7' 'B6' 'A2' 3000]
 ['B1' 'A3' 'B7' 'D6' 'B2' 3000]
 ['B1' 'A3' 'B7' 'A6' 'A2' 5000]
 ...
 ['B1' 'B3' 'D7' 'A6' 'A2' 10000]
 ['C1' 'C3' 'A7' 'C6' 'A2' 3000]
 ['D1' 'B3' 'B7' 'A6' 'A2' 10000]]


In [136]:
#划分数据集
train_data = data[:35001]
test_data = data[35001:]

In [137]:
#生成训练数据
def createDataSet():
    dataSet = train_data
    atrributes = ['experience','education','major','location','company_type']
    return dataSet, atrributes

dataSet, attributes = createDataSet()
print dataSet
print attributes

[['A1' 'A3' 'B7' 'B6' 'A2' 3000]
 ['B1' 'A3' 'B7' 'D6' 'B2' 3000]
 ['B1' 'A3' 'B7' 'A6' 'A2' 5000]
 ...
 ['D1' 'B3' 'A7' 'B6' 'A2' 5000]
 ['A1' 'C3' 'A7' 'A6' 'A2' 10000]
 ['A1' 'A3' 'B7' 'B6' 'A2' 5000]]
['experience', 'education', 'major', 'location', 'company_type']


# C4.5

In [138]:
#计算熵值
import math
def calcEnt(dataSet):
    labels = set([dataVec[-1] for dataVec in dataSet])
    labelSet = {}
    for label in labels:
        labelSet[label] = 0
    #样本总个数
    totalNum = len(dataSet)
    #类别集合
    #计算每个类别的样本个数
    for dataVec in dataSet:
        label = dataVec[-1]
        if label not in labelSet.keys():
            labelSet[label] = 0
        labelSet[label] += 1
    Ent = 0
    #计算熵值
    for key in labelSet:
        pi = float(labelSet[key])/totalNum
        Ent -= pi*math.log(pi,2)
    #print labelSet
    return Ent

In [139]:
#按给定特征划分数据集:返回第featNum个特征其值为value的样本集合，且返回的样本数据中已经去除该特征
def splitDataSet(dataSet, featNum, featvalue):
    retDataSet = []
    for dataVec in dataSet:
        if dataVec[featNum] == featvalue:
            splitData = dataVec[:featNum]
            splitData = np.append(splitData,dataVec[featNum+1:])
            retDataSet.append(splitData)
    return np.array(retDataSet)

In [140]:
#选择最好的特征划分数据集
def chooseBestFeatToSplit(dataSet, labelSet):
    featNum = len(dataSet[0]) - 1
    maxInfoGain = 0
    bestFeat = -1
    #计算样本熵值，对应公式中：H(X)
    baseEnt = calcEnt(dataSet)
    #以每一个特征进行分类，找出使信息增益最大的特征
    for i in range(featNum):
        featList = [dataVec[i] for dataVec in dataSet]
        featList = set(featList)
        #print labelSet[i],"的所有属性值",featList
        newEnt = 0
        #计算以第i个特征进行分类后的熵值，对应公式中：H(X|Y)
        for featValue in featList:
            subDataSet = splitDataSet(dataSet, i, featValue)
            prob = len(subDataSet)/float(len(dataSet))
            newEnt += prob*calcEnt(subDataSet)
        #ID3算法：计算信息增益,对应公式中：g(X,Y)=H(X)-H(X|Y)
        #infoGain = baseEnt - newEnt
        #C4.5算法：计算信息增益比
        selfClass = np.array([dataVec[i] for dataVec in dataSet]).reshape(-1,1)
        if calcEnt(selfClass) == 0:
            return i
        infoGain = (baseEnt - newEnt)/calcEnt(selfClass)
        #print labelSet[i],"的信息增益率", infoGain
        #找出最大的熵值以及其对应的特征
        if infoGain > maxInfoGain:
            maxInfoGain = infoGain
            bestFeat = i
    return bestFeat

In [141]:
#如果决策树递归生成完毕，且叶子节点中样本不是属于同一类，则以少数服从多数原则确定该叶子节点类别
def majorityCnt(labelList):
    labels = set(labelList)
    labelSet = {}
    for label in labels:
        labelSet[label] = 0
    #统计每个类别的样本个数
    for label in labelList:
        if label not in labelSet.keys():
            labelSet[label] = 0
        labelSet[label] += 1
    #iteritems：返回列表迭代器
    #operator.itemgeter(1):获取对象第一个域的值
    #True：降序
    sortedLabelSet = sorted(labelSet.items(), key=operator.itemgetter(1), reverse=True)
    return sortedLabelSet[0][0]

In [142]:
import operator

#创建决策树
def createDecideTree(dataSet, featName):
    #数据集的分类类别
    classList = [dataVec[-1] for dataVec in dataSet]
    #所有样本属于同一类时，停止划分，返回该类别
    if len(classList) == classList.count(classList[0]):
        return classList[0]
    #所有特征已经遍历完，停止划分，返回样本数最多的类别
    if len(dataSet[0]) == 1:
        return majorityCnt(classList)
    #选择最好的特征进行划分
    bestFeat = chooseBestFeatToSplit(dataSet, featName)
    bestFeatName = featName[bestFeat]
    #print "当前最佳属性是", bestFeatName
    del featName[bestFeat]
    #以字典形式表示树
    DTree = {bestFeatName:{}}
    #根据选择的特征，遍历该特征的所有属性值，在每个划分子集上递归调用createDecideTree
    featValue = [dataVec[bestFeat] for dataVec in dataSet]
    featValue = set(featValue)
    #print "剩余候选属性是", featName
    for value in featValue:
        subFeatName = featName[:]
        DTree[bestFeatName][value] = createDecideTree(splitDataSet(dataSet,bestFeat,value), subFeatName)
    return DTree

In [190]:
DT = createDecideTree(dataSet,attributes)
print DT

{'experience': {'A1': {'location': {'D6': {'company_type': {'C2': 3000, 'A2': {'education': {'C3': {'major': {'F7': 3000, 'B7': 3000, 'A7': 3000, 'D7': 3000, 'E7': 1000, 'C7': 3000}}, 'A3': {'major': {'F7': 3000, 'B7': 3000, 'A7': 3000, 'D7': 3000, 'E7': 3000, 'C7': 3000}}, 'E3': {'major': {'D7': 8000, 'B7': 8000}}, 'B3': {'major': {'F7': 3000, 'B7': 3000, 'A7': 3000, 'D7': 3000, 'E7': 3000, 'C7': 3000}}}}, 'B2': {'major': {'E7': {'education': {'C3': 3000, 'B3': 3000}}, 'B7': {'education': {'C3': 3000, 'A3': 8000, 'B3': 10000}}, 'C7': {'education': {'C3': 8000, 'A3': 1000, 'B3': 3000}}, 'A7': {'education': {'C3': 3000, 'A3': 1000, 'B3': 3000}}, 'F7': 5000}}}}, 'B6': {'education': {'C3': {'company_type': {'C2': 1000, 'A2': {'major': {'F7': 3000, 'B7': 3000, 'A7': 3000, 'D7': 3000, 'E7': 3000, 'C7': 3000}}, 'D2': 3000, 'B2': {'major': {'D7': 1000, 'B7': 3000, 'C7': 1000, 'A7': 3000}}}}, 'A3': {'company_type': {'A2': {'major': {'F7': 3000, 'B7': 3000, 'A7': 3000, 'D7': 3000, 'E7': 3000, '

In [144]:
#使用训练的模型进行分类
def classify(tree,feat,featValue):
    firstFeat = list(tree.keys())[0]
    secondDict = tree[firstFeat]
    featIndex = feat.index(firstFeat)
    key = featValue[featIndex]
    if key in secondDict.keys():
        if type(secondDict[key]).__name__ == 'dict':
            classLabel = classify(secondDict[key],feat,featValue)
        else:
            classLabel = secondDict[key]
    else:
        classLabel = "OH NO!"
    return classLabel

In [172]:
#计算测试集准确率
feat = ['experience','education','major','location','company_type']
def precisionCal(tree,feat,test_data):
    tests = [dataVec[:-1] for dataVec in test_data]
    true = 0
    sum = len(tests)
    for i in range(len(tests)):
        result = classify(tree,feat,tests[i])
        if result == "OH NO!":
            sum = sum - 1
        else:
            #print result," versus ",test_data[i][5]
            if result == test_data[i][-1]:
                true = true + 1
    return float(true)/float(sum)

In [173]:
print precisionCal(DT,feat,test_data)

0.518890675241


In [174]:
#C4.5剪枝
import copy

def isTree(obj):#判断当前节点是否是叶节点
    return (type(obj).__name__ == 'dict')

def prune(tree, testData, featList):
    newTree = copy.deepcopy(tree)
    firstFeat = list(tree.keys())[0]
    secondDict = tree[firstFeat]
    #print firstFeat
    for key in secondDict.keys():
        #print key
        if isTree(secondDict[key]):
            #print "is tree"
            #print secondDict[key]
            featNum = featList.index(firstFeat)
            subSample = splitDataSet(testData, featNum, key)
            subclassList = [dataVec[-1] for dataVec in subSample]
            if len(subclassList)!=0:
                #print "majority", majorityCnt(subclassList)
                if len(subclassList) == subclassList.count(subclassList[0]):
                    tree[firstFeat][key] = majorityCnt(subclassList)
                else:
                    newTree[firstFeat][key] = majorityCnt(subclassList)
                    precison_old = precisionCal(tree, featList, testData)
                    precision_new = precisionCal(newTree, featList, testData)
                    #print precison_old, "versus", precision_new
                    if precision_new > precison_old:
                        #print "new tree"
                        tree[firstFeat][key] = majorityCnt(subclassList)
                    else:
                        #print "go on"
                        newfeatList = featList[:featNum] + featList[featNum + 1:]
                        tree[firstFeat][key] = prune(secondDict[key], subSample, newfeatList)
            else:
                continue
    return tree  

In [175]:
import copy
tree = copy.deepcopy(DT)
atrributes = ['experience','education','major','location','company_type']
newDT = prune(tree, test_data, atrributes)

In [149]:
atrributes = ['experience','education','major','location','company_type']
precisionCal(newDT,atrributes,test_data)

0.5334535256410257

# CART

In [150]:
#划分样本集
def binarySplitDataSet(dataset,feature,value):
    matLeft = dataset[np.nonzero(dataset[:,feature] <= value)[0],:]
    matRight = dataset[np.nonzero(dataset[:,feature] > value)[0],:]
    return matLeft,matRight

In [151]:
def regressErr(dataset):
    return np.var(dataset[:,-1]) * np.shape(dataset)[0]

def chooseBestSplit(dataset, threshold=(1, 200)):
    thresholdErr = threshold[0]
    thresholdSamples = threshold[1]
    if len(set(dataset[:,-1].tolist())) == 1:
        return None,np.mean(dataset[:,-1])
    m,n = np.shape(dataset)
    Err = regressErr(dataset)
    bestErr = np.inf 
    bestFeatureIndex = 0
    bestFeatureValue = 0
    for featureindex in range(n-1):#最优划分特征选择
        #print featureindex
        for featurevalue in set(dataset[:,featureindex]):#最优划分特征值选择
            #print featurevalue
            matLeft,matRight = binarySplitDataSet(dataset,featureindex,featurevalue)
            if (np.shape(matLeft)[0] < thresholdSamples) or (np.shape(matRight)[0] < thresholdSamples):
                continue
            temErr = regressErr(matLeft) + regressErr(matRight)
            #print "temErr",temErr
            if temErr < bestErr:
                bestErr = temErr
                bestFeatureIndex = featureindex
                bestFeatureValue = featurevalue
    #检验在所选出的最优划分特征及其取值下，误差平方和与未划分时的差是否小于阈值，若是，则不适合划分
    if (Err - bestErr) < thresholdErr:
        return None,np.mean(dataset[:,-1])
    matLeft,matRight = binarySplitDataSet(dataset,bestFeatureIndex,bestFeatureValue)
    #检验在所选出的最优划分特征及其取值下，划分的左右数据集的样本数是否小于阈值，若是，则不适合划分
    if (np.shape(matLeft)[0] < thresholdSamples) or (np.shape(matRight)[0] < thresholdSamples):
        return None,np.mean(dataset[:,-1])
    return bestFeatureIndex,bestFeatureValue

In [152]:
def createCARTtree(dataset,threshold=(1,200)):
    feature,value = chooseBestSplit(dataset,threshold)
    #当不满足阈值或某一子数据集下输出全相等时，返回叶节点
    if feature == None: return value
    returnTree = {}
    returnTree['bestSplitFeature'] = feature
    returnTree['bestSplitFeatValue'] = value
    leftSet,rightSet = binarySplitDataSet(dataset,feature,value)
    returnTree['left'] = createCARTtree(leftSet,threshold)
    returnTree['right'] = createCARTtree(rightSet,threshold)
    return returnTree

In [153]:
cart_DT = createCARTtree(train_data,threshold=(1,300))
print cart_DT

{'bestSplitFeature': 0, 'bestSplitFeatValue': 'B1', 'right': {'bestSplitFeature': 0, 'bestSplitFeatValue': 'C1', 'right': {'bestSplitFeature': 3, 'bestSplitFeatValue': 'A6', 'right': {'bestSplitFeature': 0, 'bestSplitFeatValue': 'D1', 'right': 11546.703296703297, 'left': {'bestSplitFeature': 1, 'bestSplitFeatValue': 'A3', 'right': {'bestSplitFeature': 3, 'bestSplitFeatValue': 'C6', 'right': 8593.272171253822, 'left': {'bestSplitFeature': 3, 'bestSplitFeatValue': 'B6', 'right': 9386.533665835412, 'left': {'bestSplitFeature': 2, 'bestSplitFeatValue': 'A7', 'right': 9289.473684210527, 'left': 9099.514563106795}}}, 'left': {'bestSplitFeature': 2, 'bestSplitFeatValue': 'A7', 'right': 7508.87573964497, 'left': 7426.751592356688}}}, 'left': {'bestSplitFeature': 1, 'bestSplitFeatValue': 'A3', 'right': {'bestSplitFeature': 2, 'bestSplitFeatValue': 'A7', 'right': {'bestSplitFeature': 2, 'bestSplitFeatValue': 'B7', 'right': 11502.732240437159, 'left': 11117.936117936119}, 'left': 13829.6836982968

In [154]:
#使用模型进行预测
def isTree(obj):#判断当前节点是否是叶节点
    return type(obj).__name__ == 'dict'

def treeForeCast(tree, inputData):
    if not isTree(tree): return float(tree)
    if inputData[tree['bestSplitFeature']] <= tree['bestSplitFeatValue']:
        if isTree(tree['left']):
            return treeForeCast(tree['left'],inputData)
        else:
            return float(tree['left'])
    else:
        if isTree(tree['right']):
            return treeForeCast(tree['right'],inputData)
        else:
            return float(tree['right'])

In [182]:
#计算测试集准确率
def tranverse(data):
    if 1000<=data<3000:
        data = 1000
    if 3000<=data<5000:
        data = 3000
    if 5000<=data<8000:
        data = 5000
    if 8000<=data<10000:
        data = 8000
    if 10000<=data<40000:
        data = 10000
    if data>=40000:
        data = 40000
    return data

def precisionCal(tree,test_data):
    tests = [dataVec[:-1] for dataVec in test_data]
    true = 0
    sum = len(tests)
    for i in range(len(tests)):
        result = treeForeCast(tree, tests[i])
        #print result," versus ",test_data[i][-1]
        if abs(result - test_data[i][-1]) <= 2000:
            true = true + 1
    return float(true)/float(sum)

In [183]:
precisionCal(cart_DT,test_data)

0.6199239847969594

In [184]:
#CART剪枝
def getMean(tree):
    if isTree(tree['left']): tree['left'] = getMean(tree['left'])
    if isTree(tree['right']): tree['right'] = getMean(tree['right'])
    return (tree['left'] + tree['right'])/2.0

def cart_prune(tree, testData):
    if np.shape(testData)[0] == 0: return getMean(tree)#存在测试集中没有训练集中数据的情况
    if isTree(tree['left']) or isTree(tree['right']):
        leftTestData, rightTestData = binarySplitDataSet(testData,tree['bestSplitFeature'],tree['bestSplitFeatValue'])
    #递归调用prune函数对左右子树,注意与左右子树对应的左右子测试数据集
    if isTree(tree['left']): tree['left'] = cart_prune(tree['left'],leftTestData)
    if isTree(tree['right']): tree['right'] = cart_prune(tree['right'],rightTestData)
    #当递归搜索到左右子树均为叶节点时，计算测试数据集的误差平方和
    if not isTree(tree['left']) and not isTree(tree['right']):
        leftTestData, rightTestData = binarySplitDataSet(testData,tree['bestSplitFeature'],tree['bestSplitFeatValue'])
        errorNOmerge = sum(np.power(leftTestData[:,-1] - tree['left'],2)) +sum(np.power(rightTestData[:,-1] - tree['right'],2))
        errorMerge = sum(np.power(testData[:,-1] - getMean(tree),2))
        if errorMerge < errorNOmerge:
            return getMean(tree)
        else: return tree
    else: return tree

In [185]:
#计算剪枝后准确率
import copy

cart_tree = copy.deepcopy(cart_DT)
cart_tree = cart_prune(cart_tree, test_data)
precisionCal(cart_tree,test_data)

0.6263252650530106

# Bagging

In [176]:
#生成C4.5随机森林
Forest = []

def createSample():
    index = np.random.randint(35000, size=30000)
    train_sample = train_data[index,:]
    atrributes = ['experience','education','major','location','company_type']
    return train_sample, atrributes

def createForest():
    for i in range(7):
        print "creating tree",i,"..."
        train_sample, attributes = createSample()
        DT_sample = createDecideTree(train_sample,attributes)
        atrributes = ['experience','education','major','location','company_type']
        DT_sample = prune(DT_sample, train_sample, atrributes)
        Forest.append(DT_sample)  

In [177]:
createForest()

creating tree 0 ...
creating tree 1 ...
creating tree 2 ...
creating tree 3 ...
creating tree 4 ...
creating tree 5 ...
creating tree 6 ...


In [178]:
def tranverse(data):
    if 1000<=data<3000:
        data = 1000
    if 3000<=data<5000:
        data = 3000
    if 5000<=data<8000:
        data = 5000
    if 8000<=data<10000:
        data = 8000
    if 10000<=data<40000:
        data = 10000
    if data>=40000:
        data = 40000
    return data

def precisionForest(Forest,feat,test_data):
    tests = [dataVec[:-1] for dataVec in test_data]
    true = 0
    sum = len(tests)
    for i in range(len(tests)):
        results = []
        for j in range(len(Forest)):
            result = classify(Forest[j],feat,tests[i])
            if result != "OH NO!":
                results.append(result)
        #print results
        if len(results)>0:
            forestResult = majorityCnt(results)
            #print forestResult,"versus",test_data[i][-1]
            if forestResult == test_data[i][-1]:
                true = true + 1
        else:
            sum = sum - 1
    return float(true)/float(sum)
    

In [179]:
attributes = ['experience','education','major','location','company_type']
precisionForest(Forest,attributes,test_data)

0.5145319703347364

In [186]:
cart_Forest = []

def createSample():
    index = np.random.randint(35000, size=30000)
    train_sample = train_data[index,:]
    atrributes = ['experience','education','major','location','company_type']
    return train_sample, atrributes

def createCARTForest():
    for i in range(7):
        print "creating tree",i,"..."
        train_sample, attributes = createSample()
        DT_sample = createCARTtree(train_sample,threshold=(1,250))
        atrributes = ['experience','education','major','location','company_type']
        DT_sample = cart_prune(DT_sample, train_sample)
        cart_Forest.append(DT_sample)  

In [187]:
createCARTForest()

creating tree 0 ...
creating tree 1 ...
creating tree 2 ...
creating tree 3 ...
creating tree 4 ...
creating tree 5 ...
creating tree 6 ...


In [188]:
def precisionCARTForest(Forest,feat,test_data):
    tests = [dataVec[:-1] for dataVec in test_data]
    true = 0
    sum = len(tests)
    for i in range(len(tests)):
        results = []
        for j in range(len(Forest)):
            result = treeForeCast(cart_Forest[j], tests[i])
            results.append(result)
        #print results
        if len(results)>0:
            forestResult = np.mean(results)
            #print forestResult,"versus",test_data[i][-1]
            if abs(forestResult - test_data[i][-1]) <= 2500:
                true = true + 1
        else:
            sum = sum - 1
    return float(true)/float(sum)

In [189]:
attributes = ['experience','education','major','location','company_type']
precisionCARTForest(cart_Forest,attributes,test_data)

0.7181436287257451